In [11]:
import paramiko
from os.path import exists
import os
from getpass import getpass

# Checks to see if the results.body file already exists.
# if the file does exist then the user has the option to delete it.
# the user can also ignore the warnings and append to the existing file.
if os.path.exists("results.body"):
    user_input = input("The file already exists. Enter 'y' to delete the file or any other key to keep and continue:")
    if user_input == ('y'):
        os.remove("results.body")

# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password: ")

# Host Information
host = "192.168.6.71"
port = 2222
username = "blaise.notter"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication Failed")

In [12]:
sftp = ssh.open_sftp()

# Set fs.py to a string
script = 'kraken'

# Concat my remote home dir to the script
# Then upload the script
remote_path = '/home/blaise.notter/' + script
sftp.put(script, remote_path)

# Run the uplaoded file
command = ['sudo -S /home/blaise.notter/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin']

# Loop through the files and download them
for eachCMD in command:
    # Get the output from the command
    stdin, stdout, stderr = ssh.exec_command(eachCMD)

    # Input password to run command as sudo, including a newline
    # character that provides 'Enter' as if you were typing it on a keyboard
    stdin.write(thePass + "\n")

    # Get results from stdout
    lines = stdout.readlines()
    # print(lines)

    error = stderr.readlines()
    print(error)

    # Covert the list to a string
    output = ''.join(lines)

    print(output)
    # Write to a file
    #with open('results.body', 'a') as f:
    #    f.write(output)


ssh.close()

['[sudo] password for blaise.notter: sudo: /home/blaise.notter/kraken: command not found\n']

